<a href="https://colab.research.google.com/github/keinpyisi/Database/blob/main/CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model
import requests
import json

In [2]:
#Getting Encoder for Myanmar Chars
converter = "https://raw.githubusercontent.com/keinpyisi/Database/main/chatbot/converter.json"
response = requests.get(converter)
open("converter.json", "wb").write(response.content)

4113

In [3]:
#Getting Data To Train
rawcsv = "https://raw.githubusercontent.com/keinpyisi/Database/main/chatbot/training_data_eng_tag.csv"
response = requests.get(rawcsv)
open("training_data_eng_tag.csv", "wb").write(response.content)

33561

In [25]:
#Getting Data To Response
rawcsv = "https://raw.githubusercontent.com/keinpyisi/Database/main/chatbot/response.json"
response = requests.get(rawcsv)
open("response.json", "wb").write(response.content)

29724

In [4]:
def encode(character):
  cha=character
  
  # Opening JSON file
  f = open('converter.json')
    
  # returns JSON object as 
  # a dictionary
  data = json.load(f)

  # Iterating through the json
  # list
  for i in data:
      fromdata=i.get('from')
      if(fromdata.__eq__(character)):
        # Closing file
        f.close()
        return i.get('to')

    

In [5]:
def decode(character):
  cha=character
  
  # Opening JSON file
  f = open('converter.json')
    
  # returns JSON object as 
  # a dictionary
  data = json.load(f)

  # Iterating through the json
  # list
  for i in data:
      fromdata=i.get('to')
      if(fromdata.__eq__(character)):
        # Closing file
        f.close()
        return i.get('from')

    
  

In [24]:
print(decode('xz^#?ǄaȲȰǆhǅǆpe^!y<t^ǆy%?xǅǆ'))

None


In [7]:
from csv import reader
import json 
import csv
# open file in read mode
with open('training_data_eng_tag.csv', 'r') as read_obj:
        # name of csv file 
    filename = "training_data_notfixex.csv"
    csv_reader = reader(read_obj)
    cantencode = set() 
        
    # writing to csv file 
    with open(filename, 'w') as csvfile: 
      csv_output = csv.writer(csvfile)
      # pass the file object to reader() to get the reader object
      
      i=next(csv_reader, None)  # skip the headers
      csv_output.writerow(i)
      # Iterate over each row in the csv using reader object
      for row in csv_reader:
          # row variable is a list that represents a row in csv
          append=""
          for char in row[0]:
            if(encode(char) is None):
              cantencode.add(char)
              break
            char=encode(char)
            append+=char
          if(append is None or append==" " or append.isspace()):
            print("A")
            print(append)
            break
          else:  
             row[0]=append
             csv_output.writerow(row)
         
      
      print(cantencode)
         
        


{'\u200b'}


In [8]:
import csv

DELIMITER = ','

def valid_column(col):
    try:
        return float(col) != 0
    except ValueError:
        pass
    return len(col.strip()) > 0


def fix_csv(input_file, output_file, cols=0):
    if cols == 0:
        with open(input_file, newline='') as indata:
            cols = max(len(row) for row in csv.reader(indata, delimiter=DELIMITER))
    with open(input_file, newline='') as indata, open(output_file, 'w', newline='') as outdata:
        writer = csv.writer(outdata, delimiter=DELIMITER)
        for row in csv.reader(indata, delimiter=DELIMITER):
            if len(row) == cols:
                if all(valid_column(col) for col in row):
                    writer.writerow(row)

fix_csv('training_data_notfixex.csv', 'training_data.csv')

In [9]:
# importing training data
training_data = pd.read_csv("training_data.csv")

In [10]:
# preprocessing training data
#training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
training_data_tfidf = vectorizer.fit_transform(training_data["patterns"]).toarray()

In [11]:
# preprocessing target variable(tags)
le = LabelEncoder()
training_data_tags_le = pd.DataFrame({"tags": le.fit_transform(training_data["tags"])})
training_data_tags_dummy_encoded = pd.get_dummies(training_data_tags_le["tags"]).to_numpy()

In [12]:
# creating DNN
chatbot = Sequential()
chatbot.add(Dense(10, input_shape=(len(training_data_tfidf[0]),)))
chatbot.add(Dense(8))
chatbot.add(Dense(8))
chatbot.add(Dense(6))
chatbot.add(Dense(6))


chatbot.add(Dense(len(training_data_tags_dummy_encoded[0]), activation="softmax"))
chatbot.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# fitting DNN
chatbot.fit(training_data_tfidf, training_data_tags_dummy_encoded, epochs=50, batch_size=32)

In [14]:
# saving model file
save_model(chatbot, "chatbot")

INFO:tensorflow:Assets written to: chatbot/assets


In [15]:
# importing modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.models import load_model
import json
import random

In [16]:
# importing training data
training_data = pd.read_csv("training_data.csv")
# loading model
chatbot = load_model("chatbot")
# loading responses
#responses = json.load(open("responses.json", "r"))

In [17]:
# fitting TfIdfVectorizer with training data to preprocess inputs
#training_data["patterns"] = training_data["patterns"].str.lower()
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorizer.fit(training_data["patterns"])

TfidfVectorizer(ngram_range=(1, 2))

In [18]:
# fitting LabelEncoder with target variable(tags) for inverse transformation of predictions
le = LabelEncoder()
le.fit(training_data["tags"])

LabelEncoder()

In [19]:
# transforming input and predicting intent
def predict_tag(inp_str):
    inp_data_tfidf = vectorizer.transform([inp_str]).toarray()
    predicted_proba = chatbot.predict(inp_data_tfidf)
    encoded_label = [np.argmax(predicted_proba)]
    predicted_tag = le.inverse_transform(encoded_label)[0]
    
    return predicted_tag

In [37]:
# defining chat function
def start_chat():
    print("--------------- Myanmar Language AI Chat bot  ---------------")
    print("Type EXIT to quit...")
    while True:
        inp = input("Ask anything... : ")
        
        if inp == "EXIT":
            break
        else:
            append=""
            for char in inp:
              if(encode(char) is None):
                
                break
              char=encode(char)
              append+=char
            if inp:
                tag = predict_tag(append)
                #Return Tag
                # Opening JSON file
                f = open('response.json')
                  
                # returns JSON object as 
                # a dictionary
                responses = json.load(f)
                response = random.choice(responses[tag])
                #Search Response with Tag and Random it
                print("Predicted Topic... :",tag) 
                print("Response... : ", response)
            else:
                pass

In [38]:
# calling chat function to start chatting
start_chat()

KeyboardInterrupt: ignored